# What makes Liga happy
## 1. Pairingliste erstellen

In [9]:
import itertools
import numpy as np


boats = 6
no_teams = 18
flights = 16

boat_colors = {'light_blue', 'green', 'dark blue', 'white', 'grey', 'red'}

def is_valid_bibd(blocks, v, k, lambda_val):
    """ Check if the given blocks form a valid BIBD. """
    # Initialize a v x v incidence matrix
    incidence_matrix = np.zeros((v, v))
    
    for block in blocks:
        for pair in itertools.combinations(block, 2):
            incidence_matrix[pair[0]-1, pair[1]-1] += 1
            incidence_matrix[pair[1]-1, pair[0]-1] += 1
    
    # Check all off-diagonal elements to see if they are all lambda
    for i in range(v):
        for j in range(v):
            if i != j and incidence_matrix[i, j] != lambda_val:
                return False
    return True

def generate_bibd(v, k, lambda_val):
    """ Generate a BIBD with parameters v, k, lambda_val if possible. """
    elements = list(range(1, v+1))
    all_blocks = list(itertools.combinations(elements, k))
    valid_designs = []

    # Attempt to find a valid set of blocks that forms a BIBD
    for blocks in itertools.combinations(all_blocks, v):
        if is_valid_bibd(blocks, v, k, lambda_val):
            valid_designs.append(blocks)
            break  # Found a valid design; you might want to find more

    return valid_designs


## 2. Ergebnislisten erstellen
### Ergebnislisten vorbereiten

In [19]:
import pandas as pd
max_race_columns = 16
race_columns = ['Flight {}'.format(i) for i in range(1,max_race_columns+1)]
teams = ['ASVW', 'BYC(BA)', 'BYC(BE)', 'BYC(SH)', 'BYCÜ', 'DYC', 'FSC', 'JSC', 'KYC(BW)', 'KYC(SH)', 'MSC', 'MYC', 'NRV', 'RSN', 'SMCÜ', 'SV03', 'SVI', 'VSaW', 'WYC']

number_of_boats = 6
number_of_teams = len(teams)

result_df = pd.DataFrame(index=range(number_of_teams), columns=race_columns)
result_df.index = teams
result_df['Total'] = 0
print(result_df)

        Flight 1 Flight 2 Flight 3 Flight 4 Flight 5 Flight 6 Flight 7  \
ASVW         NaN      NaN      NaN      NaN      NaN      NaN      NaN   
BYC(BA)      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
BYC(BE)      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
BYC(SH)      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
BYCÜ         NaN      NaN      NaN      NaN      NaN      NaN      NaN   
DYC          NaN      NaN      NaN      NaN      NaN      NaN      NaN   
FSC          NaN      NaN      NaN      NaN      NaN      NaN      NaN   
JSC          NaN      NaN      NaN      NaN      NaN      NaN      NaN   
KYC(BW)      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
KYC(SH)      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
MSC          NaN      NaN      NaN      NaN      NaN      NaN      NaN   
MYC          NaN      NaN      NaN      NaN      NaN      NaN      NaN   
NRV          NaN      NaN      NaN    

Eine Wettfahrt hinzufügen

In [20]:
def count_values(row):
    # You can adjust this list based on the values you're interested in
    values_of_interest = [i for i in range(1, 6+2)]
    # TODO look only in Race{}.format() columns
    counts = {value: (row == value).sum() for value in values_of_interest}
    return pd.Series(counts)
    
def add_race_results(result_df, race, results):
    for team in result_df.index:
        result_df.loc[team,'Flight {}'.format(race)] = results[team]
    return result_df

def sort_results(result_df):
    result_df['Total'] = result_df.sum(axis=1)
    counts_df = result_df.apply(count_values, axis=1)
    result_df = pd.concat([result_df, counts_df], axis=1,)
    
    sort_column_list = ['Total']
    sort_column_list.extend([i for i in range(1,number_of_boats+2)])
    sort_column_list.extend(['Flight {}'.format(i) for i in range(max_race_columns, 1, -1)])
    
    sort_column_order_list = [True]
    sort_column_order_list.extend([False for i in range(1,number_of_boats+2)])
    sort_column_order_list.extend([True for i in range(max_race_columns, 1, -1)])
    
    result_df.sort_values(by=sort_column_list, ascending=sort_column_order_list, inplace=True)
    
    for i in range(1, number_of_boats+2):
        result_df.drop(i, axis=1, inplace=True)
    result_df['Rank'] = range(1,result_df.shape[0]+1)
    return result_df

In [21]:
results = {'MYC': 2, 'JSC': 1, 'ASVW': 3, 'SV03': 2, 'TS': 2, 'AS': 3, 'HJBK': 2, 'WSVA': 1, 'KNZRV': 3, 'DSK': 2, 'MSF': 1, 'YCCh': 3, 'SCM': 2, 'CNA': 1, 'YCS': 3, 'RS280': 2, 'RCB': 1}
results2 = {'MYC': 1, 'JSC': 2, 'ASVW': 3, 'SV03': 1, 'TS': 2, 'AS': 3, 'HJBK': 1, 'WSVA': 2, 'KNZRV': 3, 'DSK': 1, 'MSF': 2, 'YCCh': 3, 'SCM': 1, 'CNA': 2, 'YCS': 3, 'RS280': 1, 'RCB': 2}
results3 = {'MYC': 3, 'JSC': 2, 'ASVW': 1, 'SV03': 3, 'TS': 1, 'AS': 1, 'HJBK': 3, 'WSVA': 2, 'KNZRV': 1, 'DSK': 3, 'MSF': 2, 'YCCh': 1, 'SCM': 3, 'CNA': 2, 'YCS': 1, 'RS280': 3, 'RCB': 2}

In [22]:
result_df = add_race_results(result_df, 1, results)
result_df = add_race_results(result_df, 2, results2)
result_df = add_race_results(result_df, 3, results3)
result_df = sort_results(result_df)
print(result_df)

KeyError: 'BYC(BA)'

In [14]:
list1 = [1,2,3]
list1.extend([True for i in range(3)]).extend([False for i in range(3)])
print(list1)

AttributeError: 'NoneType' object has no attribute 'extend'

In [ ]:

# Sample DataFrame construction
data = {
    'Boat A': [2, 2, 2, 1],
    'Boat B': [1, 1, 3, 2],
    'Boat C': [2, 1, 3, 1]
}
df = pd.DataFrame(data).T

def count_values(row):
    # You can adjust this list based on the values you're interested in
    values_of_interest = [1, 2, 3]
    counts = {value: (row == value).sum() for value in values_of_interest}
    return pd.Series(counts)

df.columns = ['Race1', 'Race2', 'Race3', 'Race4']

# Step 1: Calculate total scores for each boat
df['Total'] = df.sum(axis=1)

# Step 2: Sort by total score initially
df.sort_values('Total', inplace=True)

counts_df = df.apply(count_values, axis=1)
df = pd.concat([df, counts_df], axis=1,)

df.sort_values(by=['Total',1,2,3,'Race4'],ascending=[False,False,False,False,True], inplace=True)
df.sort_values(by='Total',ascending=False).sort_values(by=[1,2,3], ascending=False).sort_values(by=['Race{}'.format(i) for i in range(max_race_columns, 1, -1)],ascending=True)

In [ ]:
print(list(df.columns[:-1])[::-1])

df['Total+1'] = list(df.columns[:-1])[::-1]
print(df)


# Step 3: Further tie-breaking
# First by best to worst race scores
sorted_df = df.sort_values(['Total'] + list(df.columns[:-1])[::-1], ascending=[True] + [True] * len(df.columns[:-1]))

# Step 4: Tie-breaking by scores in last races
#for col in reversed(df.columns[:-1]):
#    sorted_df = sorted_df.sort_values(['Total'] + list(reversed(df.columns[:-1])), ascending=True)

print(sorted_df[['Total']])

In [ ]:
# Example parameters (small scale for demonstration)
v = 4  # Number of elements (teams)
k = 2  # Number of elements per block (teams per race)
lambda_val = 1  # Each pair should appear exactly once

# Generate the BIBD
bibd_designs = generate_bibd(v, k, lambda_val)
for design in bibd_designs:
    print("Valid BIBD Design:")
    for block in design:
        print(block)

In [ ]:
import numpy as np

def round_robin_schedule(n):
    """
    Generate a round-robin schedule for n teams.
    """
    if n % 2 != 0:
        n += 1  # If odd number of teams, add a dummy team

    schedule = []
    teams = list(range(n))

    for round in range(n - 1):
        round_matches = []
        for i in range(n // 2):
            if teams[i] != n - 1 and teams[n - 1 - i] != n - 1:
                round_matches.append((teams[i], teams[n - 1 - i]))
        teams = teams[:1] + teams[-1:] + teams[1:-1]
        schedule.append(round_matches)
    
    return schedule

def assign_boats_to_races(schedule, num_boats):
    """
    Assign boats to races such that the distribution is balanced.
    """
    num_rounds = len(schedule)
    num_races_per_round = len(schedule[0])
    boat_assignments = np.zeros((num_rounds, num_races_per_round), dtype=int)

    for round in range(num_rounds):
        for race in range(num_races_per_round):
            boat_assignments[round, race] = (round + race) % num_boats
    
    return boat_assignments

def generate_racing_schedule(num_teams, num_boats):
    """
    Generate a balanced racing schedule for num_teams teams and num_boats boats.
    """
    # Step 1: Generate round-robin schedule
    schedule = round_robin_schedule(num_teams)

    # Step 2: Assign boats to races
    boat_assignments = assign_boats_to_races(schedule, num_boats)
    
    racing_schedule = []
    for round_index, round_matches in enumerate(schedule):
        for race_index, match in enumerate(round_matches):
            team1, team2 = match
            boat = boat_assignments[round_index, race_index]
            racing_schedule.append((team1, team2, boat))
    
    return racing_schedule

# Parameters
num_teams = 18
num_boats = 6

# Generate schedule
racing_schedule = generate_racing_schedule(num_teams, num_boats)

# Print schedule
for race in racing_schedule:
    print(f"Team {race[0] + 1} vs Team {race[1] + 1} on Boat {race[2] + 1}")



In [4]:
import pandas as pd
import re

data = """
Flight Race Boat 1 Boat 2 Boat 3 Boat 4 Boat 5 Boat 6
1 KYC(SH) BYCÜ SV03 NRV MSC SVI
2 VSaW FSC DYC BYC(BE) WYC JSC
3 KYC(BW) RSN MYC SMCÜ BYC(BA) ASVW
4 KYC(BW) SV03 MYC JSC SVI BYC(BE)
5 RSN BYC(BA) MSC FSC DYC NRV
6 ASVW KYC(SH) SMCÜ WYC BYCÜ VSaW
7 SV03 KYC(SH) FSC BYC(BA) MYC VSaW
8 SVI MSC BYC(BE) RSN ASVW WYC
9 SMCÜ JSC NRV DYC KYC(BW) BYCÜ
10 BYC(BE) ASVW NRV KYC(SH) KYC(BW) FSC
11 BYCÜ DYC RSN SVI VSaW MYC
12 WYC SMCÜ BYC(BA) MSC JSC SV03
13 BYC(BE) SMCÜ VSaW RSN NRV SV03
14 FSC MYC BYCÜ ASVW JSC MSC
15 BYC(BA) KYC(BW) SVI WYC KYC(SH) DYC
16 MSC KYC(BW) ASVW VSaW SV03 DYC
17 JSC BYC(BE) KYC(SH) BYCÜ RSN BYC(BA)
18 MYC NRV WYC SVI FSC SMCÜ
19 SV03 RSN WYC BYCÜ FSC KYC(BW)
20 NRV SVI JSC BYC(BA) VSaW ASVW
21 DYC BYC(BE) MSC MYC SMCÜ KYC(SH)
22 DYC SV03 RSN JSC ASVW KYC(SH)
23 FSC BYC(BA) SVI SMCÜ BYC(BE) BYCÜ
24 MSC VSaW KYC(BW) NRV WYC MYC
25 SMCÜ ASVW DYC SV03 SVI FSC
26 WYC BYCÜ BYC(BA) MYC BYC(BE) NRV
27 JSC MSC KYC(BW) VSaW KYC(SH) RSN
28 NRV WYC SV03 ASVW MYC RSN
29 SVI JSC KYC(SH) KYC(BW) SMCÜ FSC
30 BYC(BA) DYC VSaW MSC BYCÜ BYC(BE)
31 BYC(BA) NRV FSC MSC RSN JSC
32 MYC VSaW BYC(BE) KYC(BW) SV03 SVI
33 BYCÜ WYC ASVW KYC(SH) DYC SMCÜ
34 BYCÜ BYC(BE) ASVW SV03 BYC(BA) KYC(BW)
35 VSaW SVI SMCÜ FSC RSN MSC
36 KYC(SH) MYC JSC DYC NRV WYC
37 SMCÜ MYC JSC BYCÜ MSC SV03
38 ASVW NRV SVI KYC(SH) VSaW BYC(BA)
39 RSN FSC DYC BYC(BE) KYC(BW) WYC
40 MSC FSC KYC(SH) BYC(BE) ASVW MYC
41 JSC VSaW BYC(BA) WYC SMCÜ KYC(BW)
42 SV03 RSN BYCÜ DYC NRV SVI
43 WYC JSC BYC(BE) ASVW MSC SVI
44 DYC BYC(BA) KYC(BW) RSN MYC SMCÜ
45 FSC KYC(SH) SV03 VSaW BYCÜ NRV
46 RSN KYC(SH) SV03 SVI WYC BYC(BA)
47 KYC(BW) BYCÜ SMCÜ NRV BYC(BE) MSC
48 MYC ASVW VSaW JSC FSC DYC
"""

# Parse the data
lines = data.strip().split('\n')
#headers = re.split(r'\s+', lines[0])
parsed_data = []

for line in lines[1:]:
    split_line = line.split()
    race = split_line[0].strip()
    boats = split_line[1:]
    temp_list = [int(race)]
    for boat_number, team in enumerate(boats, start=1):
        temp_list.append(team)
    parsed_data.append(temp_list)

# parsed_data

In [5]:
# Create a DataFrame
columns = ['Race']
columns.extend(['Boat{}'.format(i) for i in range(1,7)])
df = pd.DataFrame(parsed_data, columns=columns)
df['flight'] = [number for number in range(1, 17) for _ in range(3)]

df

,Race,Boat1,Boat2,Boat3,Boat4,Boat5,Boat6,flight
0,1,KYC(SH),BYCÜ,SV03,NRV,MSC,SVI,1
1,2,VSaW,FSC,DYC,BYC(BE),WYC,JSC,1
2,3,KYC(BW),RSN,MYC,SMCÜ,BYC(BA),ASVW,1
3,4,KYC(BW),SV03,MYC,JSC,SVI,BYC(BE),2
4,5,RSN,BYC(BA),MSC,FSC,DYC,NRV,2
5,6,ASVW,KYC(SH),SMCÜ,WYC,BYCÜ,VSaW,2
6,7,SV03,KYC(SH),FSC,BYC(BA),MYC,VSaW,3
7,8,SVI,MSC,BYC(BE),RSN,ASVW,WYC,3
8,9,SMCÜ,JSC,NRV,DYC,KYC(BW),BYCÜ,3
9,10,BYC(BE),ASVW,NRV,KYC(SH),KYC(BW),FSC,4
